Test Place Recognition and Hierarchical Localization on the ITLP-Campus dataset using `opr.pipelines`

In [1]:
import itertools
import shutil
from pathlib import Path

import faiss
import numpy as np
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

from opr.datasets.itlp import ITLPCampus
from opr.models.place_recognition import MinkLoc3D
from opr.pipelines.place_recognition import PlaceRecognitionPipeline


ModuleNotFoundError: No module named 'opr'

# Outdoor

## MinkLoc3D

### Prepare databases

In [2]:
TRACK_LIST = [
    "00_2023-02-10",
    "01_2023-02-21",
    "02_2023-03-15",
    "03_2023-04-11",
    "04_2023-04-13",
]

WEIGHTS_PATH = "/home/docker_opr/OpenPlaceRecognition/weights/place_recognition/minkloc3d_nclt.pth"


In [3]:
dataloaders = {}

for track in TRACK_LIST:
    dataset = ITLPCampus(
        dataset_root=f"/home/docker_opr/Datasets/ITLP_Campus/ITLP_Campus_outdoor/{track}",
        sensors=["lidar"],
        mink_quantization_size=0.5,
        max_point_distance=40.0,
        load_semantics=False,
        load_text_descriptions=False,
        load_text_labels=False,
        load_aruco_labels=False,
        indoor=False,
    )
    dataloaders[track] = DataLoader(
        dataset, batch_size=16, shuffle=False, num_workers=4, collate_fn=dataset.collate_fn
    )


In [4]:
model = MinkLoc3D()
model.load_state_dict(torch.load(WEIGHTS_PATH))
model = model.to("cuda")
model.eval();


In [8]:
for track in TRACK_LIST:
    descriptors = []
    with torch.no_grad():
        for batch in tqdm(dataloaders[track]):
            batch = {k: v.to("cuda") for k, v in batch.items()}
            final_descriptor = model(batch)["final_descriptor"]
            descriptors.append(final_descriptor.detach().cpu().numpy())
    descriptors = np.concatenate(descriptors, axis=0)

    index = faiss.IndexFlatL2(descriptors.shape[1])
    index.add(descriptors)
    Path(f"/home/docker_opr/Datasets/ITLP_Campus/ITLP_Campus_outdoor/databases/{track}").mkdir(
        parents=True, exist_ok=True
    )
    faiss.write_index(
        index, f"/home/docker_opr/Datasets/ITLP_Campus/ITLP_Campus_outdoor/databases/{track}/index.faiss"
    )

    shutil.copy(
        f"/home/docker_opr/Datasets/ITLP_Campus/ITLP_Campus_outdoor/{track}/track.csv",
        f"/home/docker_opr/Datasets/ITLP_Campus/ITLP_Campus_outdoor/databases/{track}/track.csv",
    )


100%|██████████| 40/40 [00:01<00:00, 36.77it/s]


# Test PlaceRecognitionPipeline

In [3]:
from typing import Tuple
import numpy as np
from scipy.spatial.transform import Rotation

def pose_to_matrix(pose):
    position = pose[:3]
    orientation_quat = pose[3:]
    rotation = Rotation.from_quat(orientation_quat)
    pose_matrix = np.eye(4)
    pose_matrix[:3,:3] = rotation.as_matrix()
    pose_matrix[:3,3] = position
    return pose_matrix


def compute_error(estimated_pose, gt_pose):
    estimated_pose = pose_to_matrix(estimated_pose)
    gt_pose = pose_to_matrix(gt_pose)
    error_pose = np.linalg.inv(estimated_pose) @ gt_pose
    dist_error = np.sum(error_pose[:3, 3]**2) ** 0.5
    r = Rotation.from_matrix(error_pose[:3, :3])
    rotvec = r.as_rotvec()
    angle_error = (np.sum(rotvec**2)**0.5) * 180 / np.pi
    angle_error = abs(90 - abs(angle_error-90))
    return dist_error, angle_error


In [4]:
ij_permutations = list(itertools.permutations(range(len(TRACK_LIST)), 2))

median_dist_errors = []
median_angle_errors = []
mean_dist_errors = []
mean_angle_errors = []

for i, j in tqdm(ij_permutations[:1], position=0):
    local_dist_errors = []
    local_angle_errors = []
    database = TRACK_LIST[i]
    query = TRACK_LIST[j]

    pipeline = PlaceRecognitionPipeline(
        database_dir=f"/home/docker_opr/Datasets/ITLP_Campus/ITLP_Campus_outdoor/databases/{database}",
        model=MinkLoc3D(),
        model_weights_path=WEIGHTS_PATH,
        device="cuda",
    )

    query_dataset = ITLPCampus(
        dataset_root=f"/home/docker_opr/Datasets/ITLP_Campus/ITLP_Campus_outdoor/{query}",
        sensors=["lidar"],
        mink_quantization_size=0.5,
        max_point_distance=40.0,
        load_semantics=False,
        load_text_descriptions=False,
        load_text_labels=False,
        load_aruco_labels=False,
        indoor=False,
    )

    for sample in tqdm(query_dataset, position=1):
        out = pipeline.infer(sample)
        dist_error, angle_error = compute_error(out["pose"], sample["pose"].numpy())
        local_dist_errors.append(dist_error)
        local_angle_errors.append(angle_error)

    median_dist_errors.append(np.median(local_dist_errors))
    median_angle_errors.append(np.median(local_angle_errors))
    mean_dist_errors.append(np.mean(local_dist_errors))
    mean_angle_errors.append(np.mean(local_angle_errors))


100%|██████████| 1/1 [00:35<00:00, 35.67s/it]


In [5]:
median_dist_errors, mean_dist_errors


([2.7146281571568993], [16.442354230465188])

In [6]:
median_angle_errors, mean_angle_errors


([5.768895097717426], [12.568010270903558])